In [20]:
print('all ok')

all ok


In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
from langchain_core.documents import Document
from typing import TypedDict, List

In [23]:
# Define your documents
docs = [
    Document(
        page_content="We provide AI-powered business automation named 'Emon', internal tools, and intelligent chat systems for companies.",
        metadata={"topic": "services"}
    ),
    Document(
        page_content="Project pricing usually ranges between $5,000 and $25,000 depending on complexity, integrations, and scope.",
        metadata={"topic": "pricing"}
    ),
    Document(
        page_content="API integrations may increase cost depending on security, data volume, and third-party dependencies.",
        metadata={"topic": "integration"}
    ),
    Document(
        page_content="Typical project timelines range from 4 to 8 weeks including discovery, development, and testing.",
        metadata={"topic": "timeline"}
    ),
    Document(
        page_content="We integrate with CRMs, internal APIs, databases, and cloud services using secure authentication.",
        metadata={"topic": "tech"}
    ),
]

In [24]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [25]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [26]:
vectorstore = Chroma.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [27]:
from langchain_groq import ChatGroq

In [28]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.1
)

In [29]:
# Define State with memory
class ChatState(TypedDict):
    message: str
    intent_objective: str
    rag_context: str
    persona_decision: str
    response: str
    conversation_history: List[dict]

In [30]:
from langgraph.graph import StateGraph, END

In [31]:
def input_node(state):
    return {
        "message": state["message"],
        "conversation_history": state.get("conversation_history", [])
    }

In [32]:
def intent_objective_agent(state):
    # Include conversation history for context
    history_context = "\n".join([
        f"User: {h['user']}\nAssistant: {h['assistant']}" 
        for h in state.get("conversation_history", [])[-3:]
    ])
    
    prompt = f"""
You are an intent & decision-objective classifier.

Previous conversation:
{history_context}

Current user question:
{state['message']}

Analyze the full meaning, not keywords.
Return JSON:
{{
  "primary_intent": "what the user wants to understand",
  "decision_objective": "why the user is asking",
  "topics": ["pricing", "integration", "timeline", "services", "tech"],
  "confidence": 0.9
}}
"""
    res = llm.invoke(prompt)
    return {"intent_objective": res.content}

In [33]:
# def rag_agent(state):
#     retrieved_docs = retriever.invoke(state["message"])
#     context = "\n".join(d.page_content for d in retrieved_docs)
#     return {"rag_context": context}
def rag_agent(state):
    retrieved_docs = retriever.invoke(state["message"])

    seen = set()
    unique_chunks = []

    for doc in retrieved_docs:
        content = doc.page_content.strip()
        if content not in seen:
            seen.add(content)
            unique_chunks.append(content)

    context = "\n".join(unique_chunks)
    return {"rag_context": context}


In [34]:
def persona_agent(state):
    prompt = f"""
Given the user intent and objective below, decide explanation style.

Intent & objective:
{state['intent_objective']}

Return JSON:
{{
  "primary_persona": "sales or tech or general",
  "secondary_personas": ["list"]
}}
"""
    res = llm.invoke(prompt)
    return {"persona_decision": res.content}

In [35]:
def response_agent(state):
    # Include conversation history
    history_context = "\n".join([
        f"User: {h['user']}\nAssistant: {h['assistant']}" 
        for h in state.get("conversation_history", [])[-3:]
    ])
    
    prompt = f"""
You are a business AI assistant.

Previous conversation:
{history_context}

Rules:
- Use ONLY the provided context
- Use conversation history for context awareness
- Do NOT guess
- If info is missing, say you don't know

Context:
{state['rag_context']}

Current user question:
{state['message']}

Persona decision:
{state['persona_decision']}

Provide a helpful, contextual response.
"""
    res = llm.invoke(prompt)
    
    # Update conversation history
    updated_history = state.get("conversation_history", []).copy()
    updated_history.append({
        "user": state["message"],
        "assistant": res.content
    })
    
    return {
        "response": res.content,
        "conversation_history": updated_history
    }

In [36]:
# Build the graph
graph = StateGraph(ChatState)
graph.add_node("input", input_node)
graph.add_node("intent_objective", intent_objective_agent)
graph.add_node("rag", rag_agent)
graph.add_node("persona", persona_agent)
graph.add_node("response", response_agent)

graph.set_entry_point("input")
graph.add_edge("input", "intent_objective")
graph.add_edge("intent_objective", "rag")
graph.add_edge("rag", "persona")
graph.add_edge("persona", "response")
graph.add_edge("response", END)

app = graph.compile()

In [37]:
# Test with memory
state = {
    "message": "If we integrate via API, how does pricing change?",
    "conversation_history": []
}

result = app.invoke(state)
print(result)

{'message': 'If we integrate via API, how does pricing change?', 'intent_objective': '{\n  "primary_intent": "understand how pricing is affected when using API integration",\n  "decision_objective": "evaluate cost implications to decide whether to proceed with API integration",\n  "topics": ["pricing", "integration", "services", "tech"],\n  "confidence": 0.9\n}', 'rag_context': 'API integrations may increase cost depending on security, data volume, and third-party dependencies.\nProject pricing usually ranges between $5,000 and $25,000 depending on complexity, integrations, and scope.', 'persona_decision': '{\n  "primary_persona": "sales",\n  "secondary_personas": ["technical", "product", "business"]\n}', 'response': 'Integrating via API can affect the overall project cost, but the exact impact depends on a few key factors:\n\n| Factor | How it influences price |\n|--------|------------------------|\n| **Security requirements** | More robust authentication, encryption, or compliance me

In [38]:
# Test with memory
state = {
    "message": "is any name mentioned in this services?",
    "conversation_history": []
}

result = app.invoke(state)
print(result)

{'message': 'is any name mentioned in this services?', 'intent_objective': '{\n  "primary_intent": "determine whether any name is mentioned in the described services",\n  "decision_objective": "verify the presence of personal or brand names within the service details",\n  "topics": ["services"],\n  "confidence": 0.93\n}', 'rag_context': "We provide AI-powered business automation named 'Emon', internal tools, and intelligent chat systems for companies.\nWe integrate with CRMs, internal APIs, databases, and cloud services using secure authentication.", 'persona_decision': '{\n  "primary_persona": "general",\n  "secondary_personas": ["verification", "analysis"]\n}', 'response': 'Yes. The service description mentions the name **“Emon.”** No other specific names are referenced in the provided information.', 'conversation_history': [{'user': 'is any name mentioned in this services?', 'assistant': 'Yes. The service description mentions the name **“Emon.”** No other specific names are referenc